# Processing
Demonstration of Processing endpoints.

In [ ]:
import utils.DemoClient as client
import jwt
import json

## Initialise

### Client
We instantiate a client to interact with the platform. The client dynamically registers with the Authorisation Server to take part in UMA (User Managed Access) flows through which authorization is obtained for scoped access resources on behalf of the user.

In [ ]:
#-------------------------------------------------------------------------------
# Initialise client
#-------------------------------------------------------------------------------
base_domain = "demo.eoepca.org"
platform_domain = "test." + base_domain
base_url = "https://" + platform_domain
demo = client.DemoClient(base_url)
demo.register_client()
demo.save_state()

### User
User authenticates and the client receives an ID Token (JWT) that represents the user.

In [ ]:
#-------------------------------------------------------------------------------
# Authenticate as UserA and get ID Token
#-------------------------------------------------------------------------------
USER_NAME="bob"
USER_PASSWORD="defaultPWD"
user_id_token = demo.get_id_token(USER_NAME, USER_PASSWORD)

## Workspace

In [ ]:
workspace_url = "https://workspace-api." + base_domain
workspace_access_token = None

### Workspace: Get Details

In [ ]:
# Workspace - Get Details
workspace_name = "demo-user-" + USER_NAME.lower()
response, workspace_access_token = demo.workspace_get_details(workspace_url, workspace_name, id_token=user_id_token, access_token=workspace_access_token)
workspace_details = response.json()
print(json.dumps(workspace_details["storage"], indent = 2))

### Inspect S3 Bucket

In [ ]:
# Bucket details
bucket_name = workspace_details["storage"]["credentials"]["bucketname"]
s3_access = workspace_details["storage"]["credentials"]["access"]
s3_secret = workspace_details["storage"]["credentials"]["secret"]

In [ ]:
# Quick hack S3 access
import boto3

# Init S3 session for Creodias
S3_ENDPOINT = "https://cf2.cloudferro.com:8080"
session = boto3.session.Session()
s3resource = session.resource('s3', aws_access_key_id=s3_access, aws_secret_access_key=s3_secret, endpoint_url=S3_ENDPOINT)

# List bucket contents
bucket = s3resource.Bucket(bucket_name)
for obj in bucket.objects.all():
    print(' ->', obj.key)

### Upload Application Package and register it

In [ ]:
object = s3resource.Object(bucket_name, 'application-package/s-expression/s-expression-0_0_2.cwl')
result = object.put(Body=open('../data/s-expression-cwl.cwl', 'rb'))
res = result.get('ResponseMetadata')
if res.get('HTTPStatusCode') == 200:
    print('Application package uploaded successfully')
else:
    print('Application package not uploaded')

In [ ]:
resource_url= 's3://demo-user-bob/application-package/s-expression/s-expression-0_0_2.cwl'
demo.workspace_register(workspace_url, workspace_name,resource_url, id_token=user_id_token, access_token=workspace_access_token)

## ADES
The ADES provides user-specific endpoints, using a URL path prefix.

In [ ]:
# ADES URLs
ades_base_url = "http://ades.demo.eoepca.org" #"http://ades." + platform_domain
ades_wps_url = ades_base_url + "/" + USER_NAME + "/zoo"; print("ADES WPS endpoint:", ades_wps_url)
ades_proc_url = ades_base_url + "/" + USER_NAME + "/wps3"; print("ADES API Processes endpoint:", ades_proc_url)

### ADES API Processes

In [ ]:
ades_access_token = None
app_name = "s-expression-0_0_2"

#### ADES: List Processes

In [ ]:
# API Processes - List Processes
response, ades_access_token, process_ids = demo.proc_list_processes(ades_proc_url, id_token=user_id_token, access_token=ades_access_token)
print("Processes:", process_ids)

#### ADES: Deploy Application

In [ ]:
# API Processes - Deploy Application
#response, ades_access_token = demo.proc_deploy_application(ades_proc_url, "../data/app-deploy-body-atom.json", id_token=user_id_token, access_token=ades_access_token)

response, access_token = demo.proc_deploy_application(ades_proc_url, "../data/app-deploy-body-cwl-S3.json", id_token=user_id_token, access_token=ades_access_token)


In [ ]:
# Check deploy - list processes again
response, ades_access_token, process_ids = demo.proc_list_processes(ades_proc_url, id_token=user_id_token, access_token=ades_access_token)
print("Processes:", process_ids)

#### ADES: Get Application Details

In [ ]:
# API Processes - Get Application Details
response, ades_access_token = demo.proc_get_app_details(ades_proc_url, app_name, id_token=user_id_token, access_token=ades_access_token)
print(json.dumps(response.json(), indent = 2))

#### ADES: Execute Application

In [ ]:
# API Processes - Execute Application
response, ades_access_token, job_location_path = demo.proc_execute_application(ades_proc_url, app_name, "../data/app-execute-body.json", id_token=user_id_token, access_token=ades_access_token)

#### ADES: Job Status

In [ ]:
# API Processes - Job Status
response, ades_access_token, status = demo.proc_get_job_status(ades_base_url, job_location_path, id_token=user_id_token, access_token=ades_access_token)
print(json.dumps(response.json(), indent = 2))

In [ ]:
# API Processes - Job Status (keep polling for completion)
response, ades_access_token, status = demo.proc_poll_job_completion(ades_base_url, job_location_path, interval=10, id_token=user_id_token, access_token=ades_access_token)

In [ ]:
print(response.status_code, response.headers, response.text)

#### ADES: Job Result

In [ ]:
# API Processes - Job Result
response, ades_access_token, stacCatalogUri = demo.proc_get_job_result(ades_base_url, job_location_path, id_token=user_id_token, access_token=ades_access_token)
print(json.dumps(response.json(), indent = 2))

In [ ]:
result = json.loads(response.json()["outputs"][0]["value"]["inlineValue"])["StacCatalogUri"]
print(f"Result Url: {result}")
result_folder_name = result.rsplit('/', 1)[0].rsplit(':', 1)[1].replace(bucket_name+"/","")
print(f"Result foldername: {result_folder_name}")

#### ADES: List Jobs

In [ ]:
# API Processes - List Jobs
response, ades_access_token, job_ids = demo.proc_list_jobs(ades_proc_url, app_name, id_token=user_id_token, access_token=ades_access_token)
print("Jobs:", job_ids)

#### ADES: Undeploy Application

In [ ]:
# API Processes - Undeploy Application
response, ades_access_token = demo.proc_undeploy_application(ades_proc_url, app_name, id_token=user_id_token, access_token=ades_access_token)

In [ ]:
# Check undeploy - list processes
response, ades_access_token, process_ids = demo.proc_list_processes(ades_proc_url, id_token=user_id_token, access_token=ades_access_token)
print("Processes:", process_ids)

#### Workspace: Inspect results

In [ ]:
# Quick hack S3 access
import boto3

# Init S3 session for Creodias
S3_ENDPOINT = "https://cf2.cloudferro.com:8080"
session = boto3.session.Session()
s3resource = session.resource('s3', aws_access_key_id=s3_access, aws_secret_access_key=s3_secret, endpoint_url=S3_ENDPOINT)

# List bucket contents
bucket = s3resource.Bucket(bucket_name)
for obj in bucket.objects.filter(Prefix=result_folder_name):
    print(' ->', obj.key)